In [93]:
import csv
from sklearn import tree
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [94]:
def findCommunities():
    communities = []
    with open('C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv', 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities

In [154]:
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    x=[]
    y=[]
    highCrime=[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[3:len(i)-1]
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    highCrime.append(1)
                    y.append(1)
                else:
                    highCrime.append(0)
                    y.append(0)
            else:
                features = i[3:len(i)-1]
            j+=1
        y = np.array(y)
        x = np.array(x)
        
        for i in highCrime:
            print (i)
        
        return x,y,features


In [155]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

In [156]:
calcualtePercentage(Y)

(0.62719518314099343, 0.37280481685900652)

Explaination
1a) The above code snippet is used to find if ViolentCrimesPerPop is greater than 0.1 or not. If it is greater than 0.1, then highCrime/y is appended with 1 (1 means true) and if it less than 0.1, then highCrime/y is appended with 0 ( 0 means false). The positive and negative instances are 62.71 % and 37.29% respectively. 

In [120]:
def decisionTree():
    return tree.DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [121]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

In [122]:
def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)

In [123]:
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)

In [124]:
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres

In [125]:
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec

In [126]:
def find_fmeasure(prec,rec):
    tmp = []
    for i,j in zip(prec,rec):
        tmp.append(2.0*(i*j)/(i+j))
    return tmp

In [127]:
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)
    f_score =find_fmeasure(precision,recall)
    print ("F_score", f_score)

In [128]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds)
    accuracies = []
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        evaluation(y[test_ind],predictions)
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    return avg

In [129]:
distintCommunities = findCommunities()

In [140]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv'
X,Y,Features = extractData(filename,distintCommunities,addstate = False,addcommunity = False)

In [142]:
dt = decisionTree()
dt.fit(X[:1000],Y[:1000])
predictedY=dt.predict(X[1000:])


In [143]:
confusionMatrix(Y[1000:],predictedY)

array([[458,  83],
       [187, 265]])

 Explaination :
 The confusion Matrix for Y( i.e HighCrime ) can be found above.

In [157]:
evaluation(Y[1000:],predictedY)

Confusion Matrix
[[458  83]
 [187 265]]
Accuracy 0.728096676737
Precision [0.71007751937984498, 0.7614942528735632]
Recall [0.84658040665434375, 0.58628318584070793]
F_score [0.77234401349072512, 0.66249999999999998]


Explaination: Decision Tree Classifier
1-b-i) The Accuracy, Precision, Recall and F_score for decision tree are as follows. 
Accuracy = 72.8096676737 %
Precision = [0.71007751937984498, 0.7614942528735632]
Recall = [0.84658040665434375, 0.58628318584070793]

In [146]:
do_cross_validation(X[:1000],Y[:1000],dt,10)

Confusion Matrix
[[67  7]
 [15 11]]
Accuracy 0.78
Precision [0.81707317073170727, 0.61111111111111116]
Recall [0.90540540540540537, 0.42307692307692307]
F_score [0.85897435897435892, 0.5]
Confusion Matrix
[[84  7]
 [ 3  6]]
Accuracy 0.9
Precision [0.96551724137931039, 0.46153846153846156]
Recall [0.92307692307692313, 0.66666666666666663]
F_score [0.94382022471910132, 0.54545454545454553]
Confusion Matrix
[[82  8]
 [ 4  6]]
Accuracy 0.88
Precision [0.95348837209302328, 0.42857142857142855]
Recall [0.91111111111111109, 0.59999999999999998]
F_score [0.93181818181818188, 0.5]
Confusion Matrix
[[71 11]
 [ 8 10]]
Accuracy 0.81
Precision [0.89873417721518989, 0.47619047619047616]
Recall [0.86585365853658536, 0.55555555555555558]
F_score [0.88198757763975155, 0.51282051282051289]
Confusion Matrix
[[46  6]
 [21 27]]
Accuracy 0.73
Precision [0.68656716417910446, 0.81818181818181823]
Recall [0.88461538461538458, 0.5625]
F_score [0.77310924369747902, 0.66666666666666663]
Confusion Matrix
[[76 11]


0.76000000000000001

Explaination: Cross Validation
1-c-i) The 10 cross validation's accuracy, Precision, Recall and F_score can be found above.

1-c-ii) 
    

In [148]:
sumCorrect =0
for i in range(1000,len(Y)):
    if Y[i]==predictedY[i-1000]:
        sumCorrect += 1
sumCorrect*1.0/len(Y[1000:])      


0.7280966767371602

In [149]:
featureImportance = dt.feature_importances_


In [150]:
featureImp = []
for i in range(len(Features)):
    if featureImportance[i]>0:
        featureImp.append((Features[i],featureImportance[i]))


In [151]:
sorted(featureImp, key = lambda x:x[1],reverse= True)

[('PctKids2Par', 0.32278111488525474),
 ('racePctWhite', 0.082203238294688272),
 ('PctPopUnderPov', 0.048658176745843974),
 ('PctEmplProfServ', 0.035203148969698436),
 ('agePct12t21', 0.027832380853840724),
 ('PctImmigRec10', 0.024281954949955126),
 ('MedOwnCostPctIncNoMtg', 0.023247477297892609),
 ('pctWFarmSelf', 0.022717546732230896),
 ('PctSpeakEnglOnly', 0.0206660707739965),
 ('PctWOFullPlumb', 0.019341491277634685),
 ('MedRentPctHousInc', 0.018300835127601878),
 ('PctPersOwnOccup', 0.017301228449545635),
 ('population', 0.016801033848269645),
 ('MalePctNevMarr', 0.015552055031985921),
 ('PopDens', 0.015011036729113183),
 ('AsianPerCap', 0.014848184616094962),
 ('PctPersDenseHous', 0.014335974186864425),
 ('PctWorkMom', 0.01407484166733999),
 ('PctSameCity85', 0.013732721471532105),
 ('PctImmigRec8', 0.012818159375613213),
 ('HousVacant', 0.012569018829344561),
 ('agePct16t24', 0.011800674869372554),
 ('PctHousOccup', 0.01127882934958559),
 ('racePctAsian', 0.011188410716940751),


Explaination : Decision Tree Classifier
1-b-ii ) The top features for decision tree can be found above. The reason that these are the best features is because at each split point, decision tree will choose the feature which "best" splits the observations. What qualifies as best varies, but generally the split is done so that the subsequent nodes are more homogenous/pure with respect to the target. There are different ways of measuring homogeneity, for example Gini, Entropy, Chi-square. 
Considering the accuracy of this alogorithm, I think new features can be added/removed to increase the accuracy of the algorithm.